In [1]:
# San Diego
# Albuquerque
# Colorado
# Philadelphia
# Indianapolis
# Las Vegas
# Washington
# Miami
# New York
# San Francisco

### Imports

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import urllib.parse

### HTTP Request

#### store website in variable

In [3]:
website='https://www.trulia.com/CA/San_Diego/'

#### Get Request

In [4]:
response=requests.get(website)

#### Status Code

In [5]:
response.status_code

200

### Soup Object 

In [6]:
soup=BeautifulSoup(response.content,'html.parser')

In [7]:
soup

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><script>
            window.__uspapi = function(command, version, callback) {
              try {
                if (command === 'getUSPData') {
                  var cookies = document.cookie.split(';');
                  for (var i = 0; i < cookies.length; i++) {
                    var cookie = cookies[i];
                    var separatorIndex = cookie.indexOf('=');
                    separatorIndex = separatorIndex < 0 ? cookie.length : separatorIndex;
                    var cookie_name = decodeURIComponent(cookie.slice(0, separatorIndex).replace(/^\s+/, ''));
                    if (cookie_name === 'usprivacy') {
                      var uspString = decodeURIComponent(cookie.slice(separatorIndex + 1));
                      callback({ version: version, uspString: uspString }, true);
                      return;
                    }
                  }
                }
              } catch (ex) {
                

### Results

In [8]:
result_container=soup.find_all('li',{'class':'SearchResultsList__WideCell-b7y9ki-2'})

In [9]:
len(result_container)

42

### Update Results

we just want to target the elements which have the attribute 'data-testid'

In [10]:
results_update=[]

In [11]:
for r in result_container:
    if r.has_attr('data-testid'):
        results_update.append(r)

In [12]:
len(results_update)

40

### Concatenate 2 URL Parts to get absolute URL

#### URL Part 1

In [13]:
#we combine url part 1 with url part 2 in order to get the absolute url
url_part_1='https://www.trulia.com'

#### Create List for URL Part 2

In [14]:
url_part_2=[]
#loop through results
for item in results_update:
    for link in item.find_all('div',{'data-testid':'property-card-details'}):
        url_part_2.append(link.find('a').get('href'))

In [15]:
len(url_part_2)

40

#### Join Url 1 and Url 2

In [16]:
url_joined=[]
for link_2 in url_part_2:
    url_joined.append(urllib.parse.urljoin(url_part_1, link_2))

In [17]:
len(url_joined)

40

### Get Data from First Link 

In [18]:
# Address
# bedrooms
# bathrooms
# sqft
# year built
# parking
# price

#### Store first link in variable

In [19]:
first_link=url_joined[0]

#### Get Request & Soup Object

In [20]:
response=requests.get(first_link)

In [21]:
soup=BeautifulSoup(response.content,'html.parser')

In [22]:
soup

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><script>
            window.__uspapi = function(command, version, callback) {
              try {
                if (command === 'getUSPData') {
                  var cookies = document.cookie.split(';');
                  for (var i = 0; i < cookies.length; i++) {
                    var cookie = cookies[i];
                    var separatorIndex = cookie.indexOf('=');
                    separatorIndex = separatorIndex < 0 ? cookie.length : separatorIndex;
                    var cookie_name = decodeURIComponent(cookie.slice(0, separatorIndex).replace(/^\s+/, ''));
                    if (cookie_name === 'usprivacy') {
                      var uspString = decodeURIComponent(cookie.slice(separatorIndex + 1));
                      callback({ version: version, uspString: uspString }, true);
                      return;
                    }
                  }
                }
              } catch (ex) {
                

#### Address

In [23]:
soup.find('span',{'data-testid':'home-details-summary-headline'}).get_text()

'220 Encinitas Ave'

#### Bedrooms

In [24]:
soup.find('li',{'data-testid':'bed'}).get_text()

'5 Beds'

#### Bathrooms

In [25]:
soup.find('li',{'data-testid':'bath'}).get_text()

'3 Baths'

#### Sqft

In [26]:
soup.find('li',{'data-testid':'floor'}).get_text()

'1,567 sqft'

#### year built

In [27]:
soup.find('div', string="Year Built").findNext('div').get_text()

'1973'

#### parking

In [28]:
soup.find('div', string="Parking").findNext('div').get_text()

'Garage'

#### price

In [29]:
soup.find('h3',{'data-testid':'on-market-price-details'}).get_text()

'$742,900'

### Put all together and loop through all results in page 1

In [30]:
#empty lists

address=[]
beds=[]
baths=[]
area=[]
year_built=[]
parking=[]
price=[]
#loop through all joined links
for link in url_joined:
    response=requests.get(link)
    #create soup object
    soup=BeautifulSoup(response.content,'html.parser')
    #address
    try:
        address.append(soup.find('span',{'data-testid':'home-details-summary-headline'}).get_text())
    except:
        address.append('')
    #beds
    try:
        beds.append(soup.find('li',{'data-testid':'bed'}).get_text())
    except:
        beds.append('')
    #baths
    try:
        baths.append(soup.find('li',{'data-testid':'bath'}).get_text())
    except:
        baths.append('')
    #area
    try:
        area.append(soup.find('li',{'data-testid':'floor'}).get_text())
    except:
        area.append('')
    #year built
    try:
        year_built.append(soup.find('div', string="Year Built").findNext('div').get_text())
    except:
        year_built.append('')
    #parking
    try:
        parking.append(soup.find('div', string="Parking").findNext('div').get_text())
    except:
        parking.append('')
    #price
    try:
        price.append(soup.find('h3',{'data-testid':'on-market-price-details'}).get_text())
    except:
        price.append('')
    #create a dictionary with results
    output={'Address':address,'Beds':beds,'Baths':baths,'Area':area,'Year Built':year_built,'Parking':parking,'Price':price,}

In [31]:
df=pd.DataFrame(output)

In [32]:
df

,Address,Beds,Baths,Area,Year Built,Parking,Price
0,220 Encinitas Ave,5 Beds,3 Baths,"1,567 sqft",1973,Garage,"$742,900"
1,5570 Warbler Way,3 Beds,2 Baths,"1,707 sqft",1961,2 Car Garage,"$2,195,000"
2,390 San Antonio Ave #13,2 Beds,2 Baths,"1,275 sqft",1965,No Info,"$339,000"
3,1927 Pueblo St,3 Beds,2 Baths,956 sqft,1925,2 Open Spaces,"$450,000"
4,10690 Eglantine Ct,3 Beds,2 Baths,"1,448 sqft",1995,Garage,"$972,900"
5,1927 Pueblo St,3 Beds,2 Baths,956 sqft,1925,Garage,"$450,000"
6,11414 Elmstone Ct,3 Beds,3 Baths,"1,802 sqft",2000,2 Car Garage,"$1,175,000"
7,5316 Calumet Ave,4 Beds,4 Baths,"3,960 sqft",1980,2 Car Garage,"$7,925,000"
8,8451 Westmore Rd #110,2 Beds,2 Baths,866 sqft (on 3 acres),1983,No Info,"$545,900"
9,948 G St #22,1 Bed,1 Bath,650 sqft,1999,Garage,"$450,000"


## Multiple Pages - San Diego

In [ ]:
#empty lists

address=[]
beds=[]
baths=[]
area=[]
year_built=[]
parking=[]
price=[]
#url part 1
url_part_1='https://www.trulia.com'
for i in range(1,26):
    #website
    website='https://www.trulia.com/CA/San_Diego/'+str(i)+'_p/'
    #request
    response=requests.get(website)
    #soup object
    soup=BeautifulSoup(response.content,'html.parser')
    #result container
    result_container=soup.find_all('li',{'class':'SearchResultsList__WideCell-b7y9ki-2'})
    #results update
    results_update=[]
    #only results with attribute data testid
    
    for r in result_container:
        if r.has_attr('data-testid'):
            results_update.append(r)
    #relative url
    relative_url=[]
    #loop through results
    for item in results_update:
        for link in item.find_all('div',{'data-testid':'property-card-details'}):
            relative_url.append(link.find('a').get('href'))
    url_joined=[]
    for link_2 in relative_url:
        url_joined.append(urllib.parse.urljoin(url_part_1, link_2))
    for link in url_joined:
        response=requests.get(link)
        #create soup object
        soup=BeautifulSoup(response.content,'html.parser')
        #address
        try:
            address.append(soup.find('span',{'data-testid':'home-details-summary-headline'}).get_text())
        except:
            address.append('')
        #beds
        try:
            beds.append(soup.find('li',{'data-testid':'bed'}).get_text())
        except:
            beds.append('')
        #baths
        try:
            baths.append(soup.find('li',{'data-testid':'bath'}).get_text())
        except:
            baths.append('')
        #area
        try:
            area.append(soup.find('li',{'data-testid':'floor'}).get_text())
        except:
            area.append('')
        #year built
        try:
            year_built.append(soup.find('div', string="Year Built").findNext('div').get_text())
        except:
            year_built.append('')
        #parking
        try:
            parking.append(soup.find('div', string="Parking").findNext('div').get_text())
        except:
            parking.append('')
        #price
        try:
            price.append(soup.find('h3',{'data-testid':'on-market-price-details'}).get_text())
        except:
            price.append('')
        #create a dictionary with results
        output={'Address':address,'Beds':beds,'Baths':baths,'Area':area,'Year Built':year_built,'Parking':parking,'Price':price,}




In [ ]:
df=pd.DataFrame(output)